<a href="https://colab.research.google.com/github/ddenebb/IT_Academy_Final_Project/blob/V4-oriiginal/Barcelona_Air_BnB_listings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this [link ](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=1322284596)we can find a dictionary describing the dataset´s fields

In [ ]:
import pandas as pd

# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/AAA Final Project DS/Data/listings_10_September_2022.csv')

# Load the second listing file
df2 = pd.read_csv('/content/drive/MyDrive/AAA Final Project DS/Data/listings_11_December_2022.csv')

# Load the third listing file
df3 = pd.read_csv('/content/drive/MyDrive/AAA Final Project DS/Data/listings_14_March_2023.csv')

# Load the fourth listing file
df4 = pd.read_csv('/content/drive/MyDrive/AAA Final Project DS/Data/listings_10_June_2023.csv')

# Combine the dataframes into a single dataframe
combined_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

combined_df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65583 entries, 0 to 65582
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            65583 non-null  int64  
 1   listing_url                                   65583 non-null  object 
 2   scrape_id                                     65583 non-null  int64  
 3   last_scraped                                  65583 non-null  object 
 4   source                                        65583 non-null  object 
 5   name                                          65555 non-null  object 
 6   description                                   65221 non-null  object 
 7   neighborhood_overview                         38203 non-null  object 
 8   picture

In [ ]:
# # Examine the dataset's shape
combined_df.shape

(65583, 75)

The function you provided calculates the percentage of missing data for each column in the DataFrame combined_df. It prints the column name and the corresponding percentage of missing data.

To determine at which point you need to consider processing or handling the missing data, it depends on the specific requirements of your analysis and the machine learning models you plan to use.

In [ ]:
def check_missing_value(column:'str'):
    print(f'Percentage of Missing data in \033[1m"{column}"\033[0m is \033[1m{round(combined_df[column].isna().sum()/combined_df[column].shape[0] * 100, 2)} %\033[0m')
for x in combined_df.columns:
    check_missing_value(x)

Percentage of Missing data in "id" is 0.0 %
Percentage of Missing data in "listing_url" is 0.0 %
Percentage of Missing data in "scrape_id" is 0.0 %
Percentage of Missing data in "last_scraped" is 0.0 %
Percentage of Missing data in "source" is 0.0 %
Percentage of Missing data in "name" is 0.04 %
Percentage of Missing data in "description" is 0.55 %
Percentage of Missing data in "neighborhood_overview" is 41.75 %
Percentage of Missing data in "picture_url" is 0.0 %
Percentage of Missing data in "host_id" is 0.0 %
Percentage of Missing data in "host_url" is 0.0 %
Percentage of Missing data in "host_name" is 0.01 %
Percentage of Missing data in "host_since" is 0.01 %
Percentage of Missing data in "host_location" is 21.9 %
Percentage of Missing data in "host_about" is 36.28 %
Percentage of Missing data in "host_response_time" is 17.95 %
Percentage of Missing data in "host_response_rate" is 17.95 %
Percentage of Missing data in "host_acceptance_rate" is 15.79 %
Percentage of Missing data in

Typically, there is no fixed threshold for the percentage of missing data that universally applies to all situations. However, you can consider the following general guidelines:



*   Low Missing Data Percentage: If a column has a very low percentage of missing
data (e.g., less than 5%), you might choose to simply drop the rows with missing values or use imputation methods to fill in the missing values.
*   Moderate Missing Data Percentage: For columns with a moderate percentage of missing data (e.g., between 5% and 20%), you can explore more sophisticated imputation techniques, such as mean or median imputation, regression imputation, or using machine learning models to predict the missing values.

*   High Missing Data Percentage: If a column has a high percentage of missing data (e.g., more than 20% or 30%), you may consider removing the column entirely from your analysis, as the missing data might significantly impact the accuracy of the model.

It's crucial to assess the nature of missing data and the impact it may have on your analysis. You should also consider the context of the specific dataset and the potential biases that might be introduced by handling missing data in a particular way.

Before making any decisions on how to handle missing data, you may also want to analyze the patterns of missingness across different columns. If multiple columns have missing values for the same rows, it may suggest a pattern that requires special consideration.

In summary, there is no fixed threshold for handling missing data, and the approach should be driven by the nature of the dataset and the specific analysis goals. Always ensure that any imputation or handling of missing data is well-documented and carefully considered to avoid introducing biases or inaccuracies in your analysis.

In [ ]:
print(combined_df['price'].dtype)

object


In [ ]:
combined_df['price'] = combined_df['price'].str.replace('$', '').str.replace(',', '').astype(float)


<ipython-input-71-b766fbdbe8c5>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  combined_df['price'] = combined_df['price'].str.replace('$', '').str.replace(',', '').astype(float)


In [ ]:
print(combined_df['price'].dtype)

float64


In [ ]:
outlier = combined_df[(combined_df['price'] > 1000)]
outlier.count

<bound method DataFrame.count of                        id                                      listing_url  \
296                602374              https://www.airbnb.com/rooms/602374   
1059              2439285             https://www.airbnb.com/rooms/2439285   
1477              4200509             https://www.airbnb.com/rooms/4200509   
1842              5793703             https://www.airbnb.com/rooms/5793703   
1968              6666212             https://www.airbnb.com/rooms/6666212   
...                   ...                                              ...   
63382  808538407310687262  https://www.airbnb.com/rooms/808538407310687262   
63752  830950455051439372  https://www.airbnb.com/rooms/830950455051439372   
64291  851285541984761369  https://www.airbnb.com/rooms/851285541984761369   
64338  852056501470483793  https://www.airbnb.com/rooms/852056501470483793   
64725  871663364915243450  https://www.airbnb.com/rooms/871663364915243450   

            scrape_id last_scr

In [ ]:
import pandas as pd

# Step 1: Convert 'last_scraped' column to datetime for date comparison
combined_df['last_scraped'] = pd.to_datetime(combined_df['last_scraped'])

# Filter out outdated entries (e.g., listings after September 10, 2022)
cutoff_date = pd.to_datetime('2022-09-10')
combined_df = combined_df[combined_df['last_scraped'] <= cutoff_date]

# Step 2: Remove irrelevant columns (e.g., 'scrape_id', 'source', etc.)
columns_to_remove = ['scrape_id', 'source', 'host_url', 'host_thumbnail_url', "first_review", "host_neighbourhood", "neighbourhood", "bathrooms", 'calendar_updated', "last_review", "review_scores_rating" ,"review_scores_accuracy" , "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value" , "license", "reviews_per_month"]  # List all irrelevant columns
combined_df.drop(columns=columns_to_remove, inplace=True)

# Step 3: Filter extreme prices (e.g., prices more than 3 standard deviations from the mean)
mean_price = combined_df['price'].mean()
std_price = combined_df['price'].std()
threshold_price = mean_price + 3 * std_price

# Keep listings with prices within the threshold (remove outliers)
combined_df = combined_df[combined_df['price'] <= threshold_price]

# Optional: If you want to further analyze specific property types, you can filter them too
# For example, to keep only 'Entire home/apt' and 'Private room' listings
combined_df = combined_df[combined_df['room_type'].isin(['Entire home/apt', 'Private room'])]

# Finally, reset the index after filtering
combined_df.reset_index(drop=True, inplace=True)

# Now, the 'combined_df' DataFrame is cleaned and contains relevant data for further analysis.
# You can proceed with model building or other data processing tasks.


<ipython-input-74-0cca22eaf6ea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df.drop(columns=columns_to_remove, inplace=True)


In [ ]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5611 entries, 0 to 5610
Data columns (total 56 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            5611 non-null   int64         
 1   listing_url                                   5611 non-null   object        
 2   last_scraped                                  5611 non-null   datetime64[ns]
 3   name                                          5611 non-null   object        
 4   description                                   5587 non-null   object        
 5   neighborhood_overview                         3339 non-null   object        
 6   picture_url                                   5611 non-null   object        
 7   host_id                                       5611 non-null   int64         
 8   host_name                                     5611 non-null   object

In this code, the drop() function is used to remove the rows that satisfy the conditions specified in outliers. The .index attribute is used to get the index labels of the rows to be dropped. By passing these index labels to drop(), the rows are deleted from the dataframe.

After removing the rows, the index may have gaps due to the deletion. If you want to reset the index to have consecutive integer values, you can use the reset_index() function with the drop=True parameter.

In [ ]:
# Identify the rows with price > 1000
outliers = (combined_df['price'] > 1000)

# Remove the identified rows
combined_df = combined_df.drop(combined_df[outliers].index)

# Reset the index if needed
combined_df = combined_df.reset_index(drop=True)


In [ ]:
column_list = combined_df.columns.tolist()
print(column_list)

['id', 'listing_url', 'last_scraped', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_picture_url', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'instant_bookable', 'c

In [ ]:
# Select the necessary columns for prediction
selected_columns = ['id', 'last_scraped', 'price', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'bathrooms', 'amenities']
df = combined_df[selected_columns]
df.info()
df







KeyError: ignored

Convert Price to Numeric: The 'price' column is currently of object data type. To perform numerical operations and analysis, it's better to convert it to a numeric data type. Remove the dollar sign ($) and any other non-numeric characters, and convert it to a float or integer data type. For example:

Handle Missing Values in Bedrooms: The 'bedrooms' column has some missing values. You can either drop the rows with missing values or fill them with a suitable value (e.g., mean, median, or mode). If you choose to fill the missing values, you can use the fillna() method. For example, to fill with the mean:

In [ ]:
df['bedrooms'].fillna(df['bedrooms'].mean(), inplace=True)

Drop Unnecessary Columns: From the provided information, it seems that the 'bathrooms' column contains all missing values (NaN). If this column is not necessary for your analysis, you can drop it using the drop() method:

In [ ]:
df.drop('bathrooms', axis=1, inplace=True)

Parse Date: If you plan to use the 'last_scraped' column as a temporal feature, you can convert it to a datetime data type using the to_datetime() method:

In [ ]:
df['last_scraped'] = pd.to_datetime(df['last_scraped'])

Further Feature Engineering: Consider extracting additional information from features like 'amenities' or creating new features based on your domain knowledge. For example, you could extract the number of amenities or create binary columns indicating the presence of specific amenities.

In [ ]:
df['amenities_count'] = df['amenities'].apply(lambda x: len(eval(x)))
df['has_pool'] = df['amenities'].str.contains('Pool').astype(int)
df['AC'] = df['amenities'].str.contains('Air conditioning').astype(int)
# ... add more feature engineering as needed


In [ ]:
df

In this code, the 'amenities' from the dataframe df are concatenated into a single string using ' '.join(df['amenities']). Then, the string is split into a list of individual amenities using split(','), assuming that each amenity is separated by a comma.

Next, any leading or trailing whitespaces are removed from each amenity using .strip(). Finally, list(set(amenities_list)) is used to get unique amenities by converting the list into a set (which eliminates duplicates) and then back to a list.

To assign a unique code to each unique neighborhood, you can create a mapping dictionary where each unique neighborhood is associated with a unique code. Here's an example code snippet:

In this code, unique_neighborhoods stores all the unique neighborhood names from the 'neighbourhood_cleansed' column. Then, a dictionary neighborhood_code_map is created, where each unique neighborhood is mapped to a unique code using the enumerate() function.

Finally, the map() function is used to map the neighborhood names in the 'neighbourhood_cleansed' column to their respective codes from the neighborhood_code_map dictionary. The codes are then stored in a new column called 'neighborhood_code' in the dataframe df.

Now, you can use the 'neighborhood_code' column to represent neighborhoods in the heatmap or any other visualization where numerical codes are required instead of the original neighborhood names.

In [ ]:
# Get unique neighborhoods
unique_neighborhoods = df['neighbourhood_cleansed'].unique()

# Create a mapping dictionary for neighborhood codes
neighborhood_code_map = {neighborhood: code for code, neighborhood in enumerate(unique_neighborhoods)}

# Add a new column 'neighborhood_code' to the dataframe
df['neighborhood_code'] = df['neighbourhood_cleansed'].map(neighborhood_code_map)


 As I have the neighborhood code as a representative of the neighborhoods, I can remove the 'latitude' and 'longitude' columns from the dataframe:

In [ ]:
# Drop latitude and longitude columns
df.drop(['latitude', 'longitude'], axis=1, inplace=True)

In [ ]:
# Concatenate all amenities into a single string
all_amenities = ' '.join(df['amenities'])

# Split the string by commas and remove leading/trailing whitespaces
amenities_list = [amenity.strip() for amenity in all_amenities.split(',')]

# Get unique amenities
unique_amenities = list(set(amenities_list))

# Print the unique amenities
print(unique_amenities)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Explore the dataset
print(df.info())  # Display information about the dataset, such as column names and data types
print(df.describe())  # Get summary statistics of numerical columns

# Correlation analysis
correlation = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Distribution of the target variable (price)
plt.figure(figsize=(8, 6))
sns.histplot(df['price'], kde=True)
plt.title('Distribution of Price')
plt.xlabel('Price')
plt.ylabel('Count')
plt.show()

# Visualize relationships between variables
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='accommodates', y='price')
plt.title('Price vs. Number of Passengers')
plt.show()

# Continue with more exploratory analysis and visualization as per your requirements


I am observing that the distribution of prices is tightly concentrated around 0, while the maximum value in the graphic is 80000, it could indicate the presence of outliers or incorrect data in the 'price' column. To further investigate this issue:
Check for outliers: Examine the distribution of prices to identify any extreme values or outliers. Plotting a box plot can be helpful in visualizing the distribution and identifying any outliers.

In [ ]:
# Set the seaborn theme and palette
sns.set_theme(style="ticks", palette="pastel")

# Adjust figure size
plt.figure(figsize=(12, 6))

# Draw the nested boxplot
sns.boxplot(x="bedrooms", y="price",
            palette="Set2",
            data=df)
sns.despine(offset=10, trim=True)

# Adjust plot title and axes labels
plt.title('Nested Boxplot of Price by Bedrooms')
plt.xlabel('Bedrooms')
plt.ylabel('Price')

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Show the plot
plt.show()



In [ ]:
bedrooms_stats = combined_df['bedrooms'].describe()
print(bedrooms_stats)

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:


# Drop any rows with missing values in the "accommodates" column
combined_df = combined_df.dropna(subset=['accommodates'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the figure size to be 4 times wider
plt.figure(figsize=(20, 5))

# Visualize the relationship between "accommodates" and other variables using scatter plots or box plots
sns.scatterplot(data=combined_df, x='bedrooms', y='accommodates', hue='property_type')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Number of Accommodates')
plt.title('Accommodates vs. Bedrooms by Property Type')
plt.show()

# Set the figure size to be three times wider
plt.figure(figsize=(20, 5))

sns.boxplot(data=combined_df, x='room_type', y='accommodates')
plt.xlabel('Room Type')
plt.ylabel('Number of Accommodates')
plt.title('Accommodates by Room Type')
plt.show()

# Set the figure size to be three times wider
plt.figure(figsize=(20, 5))

sns.boxplot(data=combined_df, x='neighbourhood_cleansed', y='accommodates')
plt.xticks(rotation=90)
plt.xlabel('Neighborhood')
plt.ylabel('Number of Accommodates')
plt.title('Accommodates by Neighborhood')
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
combined_df = combined_df.dropna(subset=['accommodates'])

# Prepare the feature matrix (X) and target vector (y)
X = combined_df[['property_type', 'room_type', 'bedrooms']]
y = combined_df['accommodates']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define imputer to handle missing values
imputer = SimpleImputer(strategy='most_frequent')  # You can use 'mean', 'median', or 'most_frequent'

# Define column transformer to apply imputer and one-hot encoding to specific columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', imputer, ['bedrooms']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['property_type', 'room_type'])
    ],
    remainder='passthrough'  # Keeps other columns as they are
)

# Create the pipeline with preprocessing and linear regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")



In [ ]:
print(combined_df['accommodates'].describe())
